In [18]:
%%capture

%pip install nltk spacy \
 python-docx levenshtein

In [19]:
import random
from dataclasses import dataclass
from pathlib import Path
from time import sleep
from datetime import datetime
from functools import partial

import nltk
from nltk.corpus import wordnet as wn

import spacy

from docx import Document

from Levenshtein import distance as str_distance

from google.colab import drive as _drive

drive = Path('/content/drive')
_drive.mount(str(drive), force_remount=True)

Mounted at /content/drive


In [20]:
%%capture

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')

try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    import os

    os.system("python -m spacy download es_core_news_sm")

    nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [22]:
all_spanish_synsets = [ss for ss in wn.all_synsets() if ss.lemma_names('spa')]

def format_word(word) -> str:
    return str(word).strip().replace('_', ' ').capitalize()

def get_related_spanish_word(all_spanish_synsets, nowords: set[str], n_hints: int = 1):
    if not all_spanish_synsets:
        return "No Spanish synsets found."

    word_str = None
    hints = set()
    chosen_synset = None

    # Attempt to find a valid word and enough hints
    for _ in range(100):
        hints.clear()

        # Pick a random synset and a Spanish word from it
        chosen_synset = random.choice(all_spanish_synsets)
        spa_lemmas = chosen_synset.lemma_names('spa')

        if not spa_lemmas:
            continue

        word_1 = random.choice(spa_lemmas)
        word_str = format_word(word_1)

        # Skip if we've already used this word
        if word_str in nowords:
            continue

        if n_hints == 0:
            break

        # Try to find Hypernyms or Hyponyms
        related_synsets = chosen_synset.hypernyms() + chosen_synset.hyponyms()
        random.shuffle(related_synsets)

        for rel_ss in related_synsets:
            rel_lemmas = rel_ss.lemma_names('spa')
            if rel_lemmas:
                rel_word = format_word(random.choice(rel_lemmas))
                if rel_word != word_str:
                    hints.add(rel_word)
            if len(hints) >= n_hints:
                break

        # Fallback: Extract nouns from example sentences using SpaCy
        if len(hints) < n_hints:
            examples = chosen_synset.examples()
            for example in examples:
                doc = nlp(example)

                # Extract nouns that aren't the target word itself
                for token in doc:
                    if token.pos_ == "NOUN":
                        potential_hint = token.text.lower()
                        if potential_hint != word_str.lower() and potential_hint not in hints:
                            hints.add(potential_hint)

                    if len(hints) >= n_hints:
                        break
                if len(hints) >= n_hints:
                    break

        if word_str and (len(hints) >= n_hints or n_hints > 0):
            break

    return {
        "Word": word_str,
        "Hints": list(hints)[:n_hints],
        "Definition": chosen_synset.definition() if chosen_synset else None
    }
#

In [5]:
def get_closest_player(players, pl):
    for p in players:
        if p.startswith(pl):
            return p

    return sorted(
      players,
      key=partial(str_distance, pl),
    )[0]
#

In [6]:
@dataclass
class Role:
  id: int
  name: str
  description: str
  sees_word: bool
  knows_role: bool
  n_hints: int | bool
  can_vote: bool
  voted_loses: bool
  sees_last_votes: bool = False
  voting_value: int = 1

  def __hash__(self):
    return self.id

  def __eq__(self, other):
    return self.__hash__() == other.__hash__()

  def tell(self) -> str:
    return f"Eres: {self.name}\n {self.description}"

  def __repr__(self) -> str:
    return f"<Role name={self.name}>"
#

In [7]:
civil = Role(
  id=0,
  name="Civil 😎",
  description="Adivina quienes són los impostores!",
  sees_word=True,
  knows_role=True,
  n_hints=False,
  can_vote=True,
  voted_loses=False,
)
impostor = Role(
  id=1,
  name="Impostor 😈",
  description="Engaña a los demás para que piensen que eres un civil!",
  sees_word=False,
  knows_role=True,
  n_hints=True,
  can_vote=False,
  voted_loses=True,
)
clueless = Role(
  id=2,
  name="Despistado 😳",
  description="No sabes la palabra, pero no eres un impostor! Intenta no parecer sospechoso!",
  sees_word=False,
  knows_role=True,
  n_hints=False,
  can_vote=True,
  voted_loses=False,
)
king = Role(
  id=3,
  name="Rey 👑",
  description="Encuentra a los impostores! Elige de qué súbditos fiarte e imparte justícia!",
  sees_word=True,
  knows_role=True,
  n_hints=False,
  can_vote=True,
  voted_loses=False,
  voting_value=5,
)
clueless_king = Role(
  id=4,
  name="Rey despistado 👑",
  description="Encuentra a los impostores! Elige de qué súbditos fiarte e imparte justícia, pero cuidado con los impostores!",
  sees_word=False,
  knows_role=True,
  n_hints=False,
  can_vote=True,
  voted_loses=False,
  voting_value=5,
)

In [8]:
class Voting:
  voting_sep: str="#"*10
  _voting_prefix: str="Votas a: "

  def template(self) -> str:
    return f"\n{self.voting_sep}\n{self._voting_prefix}"

  def extract_vote(self, text: str) -> str | None:
    strip_vp = self._voting_prefix.strip()
    for line in text.split('\n'):
      if line.strip().startswith(strip_vp):
        return line[len(strip_vp):].strip()

  def count_votes(self, votes: dict[str, int]) -> list[str]:
    svotes = sorted(votes.items(), key=lambda t: t[1])

    max_svotes = svotes[-1][1]

    return [
      sv[0]
      for sv in svotes
      if sv[1] == max_svotes
    ]
#

In [9]:
max_vote = Voting()

In [10]:
@dataclass
class Rule:
  name: str
  text: str

  def __hash__(self):
      return self.name.__hash__()

  def __eq__(self, other):
      return self.__hash__() == other.__hash__()
#

In [11]:
mimica = Rule(
  "Mímica",
  "Sólo se permite mímica",
)
verbs = Rule(
  "Acción",
  "Sólo se permiten verbos (acciones)",
)
basic = Rule(
  "Como un niño pequeño",
  "Sólo se permiten formas básicas y colores",
)
detective = Rule(
  "Detective",
  "No dices una palabra, preguntas a otro jugador una pregunta de sí o no",
)

In [12]:
nowords = {
  "Género ptyas",
  "Género tamiasciurus",
  "Dendrocolaptidae",
  "Pomatomidae",
  "Abomaso",
}

In [15]:
def play(
  players: set[str],
  roles: dict[Role, int],
  n_hints: int=1,
  tell_roles=True,
  add_word_definition=False,
  voting: Voting=max_vote,
  base_role: Role=civil,
  max_votations: int | None=3,
  write_path: str | None=drive/"MyDrive"/"Impostor",
  vote_failure_loses: int | bool=True,
  special_rules: dict[Rule, float]=None,
  nowords: set[str]=nowords,
):
    rroles = list()
    n_voted_out = 0
    for rol, nrol in roles.items():
        rroles.extend([rol]*nrol)

        if rol.voted_loses:
            n_voted_out += 1

    assert n_voted_out != 0
    assert n_voted_out < len(players)

    rroles.extend(
      [base_role] * (
        len(players) - len(rroles)
    ))

    random.shuffle(rroles)

    pl_roles = dict(zip(
      map(str.lower, players),
      rroles
    ))

    #print(pl_roles)

    og_pl_roles = pl_roles.copy()

    game = get_related_spanish_word(nowords, n_hints=n_hints)
    word = game['Word']
    wdef = game['Definition']
    hints = game['Hints']

    if special_rules is not None:
        rules, probs = zip(*special_rules.items())

        psum = sum(probs)
        assert psum <= 1.0

        if psum < 1.0:
            rules += (None,)
            probs += (1 - sum(probs),)

        special_rule = random.choices(rules, probs)[0]
    else:
        special_rule = None

    write_path.mkdir(parents=True, exist_ok=True)

    ftimes = dict()

    for player, rol in pl_roles.items():
        doc = Document()

        doc.add_heading(f"{player.upper()}\n", 0)
        doc.add_paragraph(f"Actualizado en: {datetime.now().strftime("%Y/%m/%d %H:%M:%S")}\n")

        if tell_roles and rol.knows_role:
            doc.add_paragraph(rol.tell())
        else:
            doc.add_paragraph("Rol DESCONOCIDO\n")

        if special_rule is not None:
            doc.add_paragraph(f"Regla especial: {special_rule.name!r}\n {special_rule.text}")

        if tell_roles and rol.knows_role:
            if rol.sees_word:
                doc.add_paragraph(f"La palabra es: {word}")
                if add_word_definition:
                    doc.add_paragraph(f"Definición (inglés): {wdef}")
            else:
                doc.add_paragraph("No conoces la palabra")

            if rol.n_hints and hints:
                 if rol.n_hints is True:
                    doc.add_paragraph('Pistas:\n -'+ '\n -'.join(hints))
                 else:
                    doc.add_paragraph('Pistas:\n -'+ '\n -'.join(hints[:rol.n_hints]))

            if rol.can_vote:
                doc.add_paragraph(voting.template())
            else:
                doc.add_paragraph("\nNO puedes votar")
        else:
            if rol.sees_word:
                doc.add_paragraph(f"Tu palabra es: {word}")
            else:
                doc.add_paragraph(f"Tu palabra es: {hints[0]}")

            doc.add_paragraph(voting.template())

        pl_path = write_path / f"{player}.docx"
        doc.save(pl_path)
        ftimes[player] = pl_path.stat().st_mtime

    lplayers = set(pl_roles.keys())
    n_votations = max_votations
    while n_votations > 0 and n_voted_out > 0:
        try:
            input("Pulsa intro para revisar las votaciones\n> ")
        except KeyboardInterrupt:
            return

        pl_votes = dict()
        votes = {lp: 0 for lp in lplayers}
        success = False
        for player, rol in pl_roles.items():
            if rol.can_vote:
                pl_path = write_path / f"{player}.docx"
                if pl_path.stat().st_mtime == ftimes[player]:
                    print(f"El archivo de {player} aún no se ha actualizado")
                    sleep(3)
                    break

                pl_vote = voting.extract_vote(
                  "\n".join(map(
                    lambda par: par.text,
                    Document(pl_path).paragraphs
                  ))
                )

                #print(f"{player} voted to {pl_vote!r}")

                if not pl_vote:
                    print(f"Voto no registrado del jugador {player}")
                    break

                pl_vote = pl_vote.lower()

                if pl_vote not in lplayers:
                    pl_vote = get_closest_player(lplayers, pl_vote.lower())

                    if pl_vote not in lplayers:
                        print(f"Voto inválido del jugador {player}")
                        break

                votes[pl_vote] += rol.voting_value
                pl_votes[player] = pl_vote
        else:
            success = True

        if not success:
            continue

        voted = voting.count_votes(votes)

        if len(voted) != 1:
            print(f"La votación ha empatado a {len(voted)}")
            n_votations -= 1
            continue

        voted_rol = pl_roles.pop(voted[0])
        print(f" {voted[0].capitalize()} eliminado")
        if voted_rol.voted_loses:
            n_voted_out -= 1
        elif vote_failure_loses is True:
            print("Los impostores han ganado")
            break
        elif isinstance(vote_failure_loses, int):
            if vote_failure_loses:
                vote_failure_loses -= 1
            else:
                print("Los impostores han ganado")
                break

    if n_voted_out == 0:
        print("Los impostores han perdido")

    print("\nRoles:")
    for player, rol in og_pl_roles.items():
        print(f" {player.capitalize()}: {rol.name}")
#

In [16]:
players = {
  "Ausias",
  "Thais",
  "Yolanda",
}
n_hints = 1
roles = {
  impostor: 1,
}
srules = {
  mimica: 0.02,
  verbs: 0.02,
  basic: 0.02,
  detective: 0.01,
}

In [17]:
play(
  players,
  roles,
  n_hints=n_hints,
  special_rules=srules,
  tell_roles=True
)

Pulsa intro para revisar las votaciones
> 
El archivo de yolanda aún no se ha actualizado
Pulsa intro para revisar las votaciones
> 
ausias eliminated
Los impostores han perdido

Roles:
 thais: Civil 😎
 yolanda: Civil 😎
 ausias: Impostor 😈
